In [ ]:
# suppress qiskit 1.0 deprecation warnings:
import warnings
import numpy as np

# Suppress all warnings
warnings.filterwarnings("ignore")

# Example code that raises warnings
warnings.warn("This is a warning!")

from qiskit import QuantumCircuit, Aer, execute
import src.qcm_sim as sim
import src.compile_keys as keys

# import circuits:
import circuits.hidden_shift as hshift 

# Hidden Shift Algorithm

The hidden shift algorithm introduced in [[Roetteler, 2008]](https://arxiv.org/abs/0811.3208) shows an oracle separation between $\mathsf{P}$ and $\mathsf{BQP}$ and has since been widely used for benchmarking classical simulation algorithms for Clifford+$T$ circuits; see e.g., [[Bravyi, Gosset, 2016]](https://arxiv.org/abs/1601.07601). (See also [[Amy, Stinchcombe, 2024]](https://arxiv.org/abs/2408.02778) for efficient classical simulation of hidden shift circuits.) In this notebook we provide the background for the correctness and query complexity of this quantum algorithm and classically simulate for various parameters using our phase space tableau.

## Background

Consider a hidden shift $s \in \mathbb{F}_2^n$ and functions

$$f, f': \mathbb{F}_2^n \rightarrow \{\pm 1\}$$

where $f'$ is the  Walsh-Hadamard (WH) transform of the shifted function $f(y\oplus s)$ given by

$$f'(x)=2^{-n/2}\displaystyle \sum_{y\in \mathbb{F}_2^n}(-1)^{x\cdot y}f(y \oplus s).$$

A boolean function $f: \mathbb{F}_2^n \rightarrow {\pm 1}$ is **bent** if its WH-transform

$$W_f(x)=2^{-n/2}\displaystyle \sum_y (-1)^{x\cdot y} f(y)$$

is **constant** for all $x \in \mathbb{F}_2^n$, i.e. $|W_f(x)|=1$.

## Maiorana Family of Bent functions:
Let $n$ be even, then a Maiorana bent function is of the form

$$ f(x,y)=(-1)^{x\cdot \pi (y)+g(y)}\quad  \text{and}\quad   x,y \in \mathbb{F}_2^{n/2}$$

where $\pi:\mathbb{F}_2^n\to \mathbb{F}_2^n$ is a permuation and $g: \mathbb{F}_2^{n/2} \rightarrow \mathbb{F}_2$ is arbitrary. The WH-transform of a Maiorana bent function is given by

$
\begin{align}
\hat f(u,v)&=\frac{1}{2^{n/2}}\displaystyle \sum_{xy}(-1)^{(x,y)\cdot(u,v)}f(x,y) \\            
&=(-1)^{\pi^{-1}(u)\cdot v+g(\pi^{-1}(u))}.
\end{align}
$

## Statement of Problem:
Suppose $f$ is a bent function and $f'$ is the Fourier (WH) transform of the shifted version $f(x\oplus s)$.

**Task:** Determine the shift $s \in \mathbb{F}_2^n$ with as few queries to an oracle giving access to the functions $f,f'$ as possible.

Our goal is to construct the quantum circuit for solving the hidden shift problem.

## Quantum Solution
Let $O_f|x\rangle = f(x)|x\rangle $ and $O_{f'}|x\rangle = f'(x)|x\rangle $ for some Maiorana bent functions $f,f'$ and define the unitary

$$U :=H_n O_{f'}H_n O_f H_n\quad \text{where}\quad H_n:=H^{\otimes n}.$$

It is possible to show (see below) that $U\lvert 0\rangle = \lvert s \rangle$, where $s$ is the hidden shift. It follows that

$$ P(y|x) = |\langle y|U|x\rangle|^2 = |\langle y|s\rangle|^2 = \delta(y,s). $$

**Remark**: Since the outcome of the circuit is deterministic a single query to the quantum oracle suffices for determining the hidden shift. Roetteler showed that an exponential number of queries are required to a classical oracle, given by access to the functions $f,f'$. Thus we have exponential oracle separation between $\mathsf{P}$ and $\mathsf{BQP}$.

### Proof of quantum solution

Recall that $n$ is even and that $f$ is a Maiorana bent function so that

$$f(u,v)= (-1)^{g(u)+v\cdot \pi(u)}$$

where $u,v \in \mathbb{F}_2^{n/2}$ and $g: \mathbb{F}_2^{n/2} \rightarrow \mathbb{F}_2$ is arbitrary. The permutation $\pi$ is also arbitrary but for convenience (see e.g., [[Bravyi, Gosset, 2016]](https://arxiv.org/abs/1601.07601)) we will set $\pi=id$. We then have that

$$f(u,v)=(-1)^{g(u)+u\cdot v}\quad \text{and}\quad \hat f(u,v)=(-1)^{u\cdot v + g(v)}, $$

where the latter is the WH-transform of $f$. From this we can also compute

$$f'(u,v) = \hat f(u,v)\cdot (-1)^{u\cdot r+v\cdot r'}\quad \text{where}\quad r,r'\in \mathbb{F}_2^{n/2}\quad \text{and}\quad s=(r,r').$$


Recall that the controlled-$Z$ operation is a diagonal operator and has the action $CZ_{ij}\lvert u_1,\cdots,u_n\rangle = (-1)^{u_i u_j}\lvert u_1,\cdots,u_n\rangle$ on the computational basis. For an arbitrary Boolean function $g$ let $O_g$ be a diagonal unitary satisfying

$$O_g|u\rangle=(-1)^{g(u)}|u\rangle.$$

It then follows readily that

$$
O_f|u,v\rangle
=\left( \prod^{n/2}_{i=1}CZ_{i,i+n/2}\right )(O_g \otimes I)|u,v\rangle
= (-1)^{g(u)+u\cdot v}|u,v\rangle
$$

Similarly we have that

$$O_{f'}|u,v\rangle=f'(u,v)|u,v\rangle\quad \text{where}\quad O_{f'} = \left( \prod^{n/2}_{i=1}CZ_{i,i+n/2}\right )(I \otimes O_g) Z(s).$$

Using these expressions it is straightforward to show, using standard methods for computing the Hadamard transform, that $H_n O_{f'} H_n O_f H_n\lvert 0\rangle = \lvert s \rangle $.

## Constructing the quantum oracle

For our family of oracles we first fix positive integers $\kappa$ and $\nu\geq 3\kappa$, with $n=2\nu$ and $c=2\kappa$. The parameter $\kappa$ controls the total number of $CCZ$ gates, and thus the number of non-Clifford gates. In particular, we will use the decomposition of the Toffoli gate $CCX$ that uses $7$ $T$ gates (see e.g., [[Toffoli gate, Wikipedia]](https://en.wikipedia.org/wiki/Toffoli_gate)) from which we have that

$$CCZ = (I\otimes I \otimes H)~ CCX ~ (I\otimes I\otimes H).$$



The $\nu$-qubit diagonal unitary $O_{g}$ corresponds to the arbitrary Boolean function $g:\mathbb{F}_{2}^{\nu}\to \mathbb{F}_{2}$. For our purposes we choose $O_{g}$ to be of the form
$$
O_{g} = \prod_{j=0}^{c-1}CCZ_{j_{1},j_{2},j_{3}}\quad \text{where}\quad j_{k} = k+jc\quad (k=1,2,3).
$$

This oracle applies $CCZ$ on disjoint $3$-qubit subsets. If $n> 3\nu$ then one can perform a diagonal unitary on the remaining qubits using diagonal Clifford gates drawn from $\{Z,CZ\}$.

In [ ]:
m = 3
n = 2*m
c = int(m/3)

# Generate a random hidden shift
shift = [1 for _ in range(n)]

# create qiskit circuit
qc = hshift.hidden_shift(n,c,shift,is_clifford = False)
qc_qasm = qc.qasm()

print(qc)

In [ ]:
# Qiskit simulation
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()
counts = result.get_counts()

counts

In [ ]:
# number of samples:
number_of_t_gates = 2*7*c
epsilon = 0.1
prob_fail = 0.01
hoeffding_samples = keys.compute_hoeffding_samples(number_of_t_gates,epsilon,prob_fail)

hoeffding_samples

In [ ]:
# Shift string
print(f"\nHidden Shift: {shift}\n")

# QCM simulation
results = sim.run_qcm(qc_qasm,shots=hoeffding_samples)